In [ ]:
import scipy.stats as stats
from myst_nb import glue
import scipy.stats as stats
%config InlineBackend.figure_formats = ['svg']

# The peri-stimulus time histogram


The peri-stimulus (or peri-event) time histogram (PSTH/PETH) is one of the most prevalent analyses in systems neuroscience. Its original usage was in identifying responses of single neurons to specific events in time (where the events were action potentials), but it can also be used to analyze time series of specific behaviors, estimate connected cell pairs, and analyze any number of other ways in which streams of events might relate to other events in time.

Here, we're going to work through the process of creating a simple PSTH using Python, noting along the way how various parameter choices capture relevant statistical assumptions and tradeoffs as well as affecting the look and feel of the resulting plots.

We'll start with some sample data, shown in the raster plot below:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#set random seed
np.random.seed(12345)

In [ ]:
# time axis: 
dt = 0.001
taxis = np.arange(-0.5, 1.5, dt)
rate = dt * np.ones_like(taxis)

# in [0, 1] rate is 10Hz
up_rate = 10 * dt
rate[(0 < taxis) & (taxis < 1)] = up_rate

# generate counts
ntrials = 50
counts = stats.poisson.rvs(rate, size=(ntrials, len(rate)))

# convert to event times
trials, time_inds = counts.nonzero()
times = taxis[time_inds]

In [ ]:
# raster plot
plt.scatter(times, trials, s=10, color='k')
plt.ylabel('trial')
plt.xlabel('time (s)');

It should be obvious from this representation of the data that the firing rate goes up around 0 and down around 1 second later. In fact, these are artificial data generated from the methods of the last section by assuming that the firing rate is 1 Hz at baseline, increases to 10 Hz at 0.0, and falls again to baseline at 1.0 s. This highly contrived example &mdash; real data rarely look this clean &mdash; will allow us to examine the tradeoffs endemic in constructing the PSTH in a case where we know the ground truth.